In [147]:
import os 

In [148]:
%pwd

'F:\\Vscode\\chiken_dataset\\Chiken-Diseases-Classification'

In [149]:
os.chdir("F:\Vscode\chiken_dataset\Chiken-Diseases-Classification")

In [150]:
%pwd


'F:\\Vscode\\chiken_dataset\\Chiken-Diseases-Classification'

In [151]:
from dataclasses import dataclass
from pathlib import Path
from tensorflow.keras.callbacks import ModelCheckpoint


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [152]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [153]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [154]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [155]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join('F:\Vscode\chiken_dataset\Chiken-Diseases-Classification\artifacts'),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
    

In [156]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2023-08-03 15:30:57,228]: INFO: common:yaml file: config\config.yaml loaded successfully]
[2023-08-03 15:30:57,244]: INFO: common:yaml file: params.yaml loaded successfully]
[2023-08-03 15:30:57,247]: INFO: common:created directory at: artifacts]
[2023-08-03 15:30:57,250]: INFO: common:created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-08-03 15:30:57,252]: INFO: common:created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
